In [27]:

# Load Oxford Pets dataset from TFDS
(train_ds, test_ds), info = tfds.load('oxford_iiit_pet:3.*.*', split=['train', 'test'], with_info=True)

# Define preprocessing functions
def preprocess_image(data):
    image = tf.image.resize(data['image'], (32, 32))  # Resize image
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixel values to [0, 1]
    label = tf.image.resize(data['segmentation_mask'], (32, 32))  # Resize label
    label = tf.cast(label, tf.int32)  # Ensure labels are integers
    
    # Convert labels to one-hot categorical
    num_classes = info.features['label'].num_classes
    label = tf.one_hot(label, depth=num_classes)
    
    # Reshape labels
    label = tf.reshape(label, (32, 32, num_classes))  # Reshape to (32, 32, num_classes)
    
    return image, label

# Apply preprocessing to datasets
train_ds = train_ds.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_ds = test_ds.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)


In [28]:
for image, label in train_ds.take(1):
    print("Image shape:", image.shape)
    print("Label shape:", label.shape)

Image shape: (32, 32, 3)
Label shape: (32, 32, 37)


In [29]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model

def create_segmentation_model(input_shape, num_classes):
    # Load ResNet50 backbone
    resnet50_base = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    # Add segmentation-specific layers on top of ResNet50
    x = resnet50_base.output
    x = tf.keras.layers.UpSampling2D(size=(32, 32))(x)  # Upsample to match original image size
    x = tf.keras.layers.Conv2D(num_classes, kernel_size=(1, 1), activation='softmax')(x)

    # Create segmentation model
    model = tf.keras.models.Model(inputs=resnet50_base.input, outputs=x)
    return model

# Instantiate the model
input_shape = (32, 32, 3)  # Adjust input shape as per your dataset
num_classes = info.features['label'].num_classes
model = create_segmentation_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_11[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 8, 8, 64)    256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 8, 8, 64)    0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 8, 8, 256)    16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_block3_3_bn (BatchNormal  (None, 8, 8, 256)   1024        ['conv2_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 4, 4, 128)   512         ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 4, 4, 128)   0           ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 4, 4, 512)    66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_block3_3_bn (BatchNormal  (None, 4, 4, 512)   2048        ['conv3_block3_3_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 2, 2, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 2, 2, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 2, 2, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_block2_3_bn (BatchNormal  (None, 2, 2, 1024)  4096        ['conv4_block2_3_conv[0][0]']    
 ization) 

 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 2, 2, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 2, 2, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 2, 2, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
                                                                                                  
 conv4_blo

 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 1, 1, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 1, 1, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 1, 1, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
                                                                                                  
 conv5_blo

In [35]:
model.fit(train_ds.batch(32), epochs=50)  # Adjust number of epochs as needed

Epoch 1/50
115/115 [==============================] - 13s 111ms/step - loss: 0.8884 - accuracy: 0.6009
Epoch 2/50
115/115 [==============================] - 11s 96ms/step - loss: 0.8828 - accuracy: 0.6081
Epoch 3/50
115/115 [==============================] - 11s 99ms/step - loss: 0.8720 - accuracy: 0.6134
Epoch 4/50
115/115 [==============================] - 11s 96ms/step - loss: 0.8623 - accuracy: 0.6209
Epoch 5/50
115/115 [==============================] - 11s 95ms/step - loss: 0.8538 - accuracy: 0.6257
Epoch 6/50
115/115 [==============================] - 10s 86ms/step - loss: 0.8515 - accuracy: 0.6262
Epoch 7/50
115/115 [==============================] - 11s 95ms/step - loss: 0.8499 - accuracy: 0.6280
Epoch 8/50
115/115 [==============================] - 13s 113ms/step - loss: 0.8600 - accuracy: 0.6235
Epoch 9/50
115/115 [==============================] - 12s 106ms/step - loss: 0.8520 - accuracy: 0.6272
Epoch 10/50
115/115 [==============================] - 12s 105ms/step - loss: 0

In [38]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_ds.batch(32))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

115/115 [==============================] - 5s 39ms/step - loss: 0.9185 - accuracy: 0.5618
Test Loss: 0.9184777140617371
Test Accuracy: 0.5618018507957458


In [39]:
import numpy as np

# Define IoU (Intersection over Union) metric
def calculate_iou(y_true, y_pred):
    intersection = np.sum(np.logical_and(y_true, y_pred))
    union = np.sum(np.logical_or(y_true, y_pred))
    iou = intersection / union
    return iou

# Define Dice coefficient metric
def calculate_dice_coefficient(y_true, y_pred):
    intersection = 2.0 * np.sum(y_true * y_pred)
    dice_coefficient = intersection / (np.sum(y_true) + np.sum(y_pred))
    return dice_coefficient

# Define Pixel Accuracy metric
def calculate_pixel_accuracy(y_true, y_pred):
    pixel_accuracy = np.sum(y_true == y_pred) / (y_true.shape[0] * y_true.shape[1])
    return pixel_accuracy

# Evaluate the model on test data
iou_scores = []
dice_coefficient_scores = []
pixel_accuracy_scores = []

for image, label in test_ds.batch(32):
    # Predict segmentation masks
    pred_mask = model.predict(image)
    
    # Convert one-hot encoded labels back to categorical
    pred_mask_categorical = np.argmax(pred_mask, axis=-1)
    true_mask_categorical = np.argmax(label, axis=-1)
    
    # Calculate metrics for each sample
    for i in range(len(pred_mask_categorical)):
        iou_scores.append(calculate_iou(true_mask_categorical[i], pred_mask_categorical[i]))
        dice_coefficient_scores.append(calculate_dice_coefficient(true_mask_categorical[i], pred_mask_categorical[i]))
        pixel_accuracy_scores.append(calculate_pixel_accuracy(true_mask_categorical[i], pred_mask_categorical[i]))

# Calculate mean scores
mean_iou = np.mean(iou_scores)
mean_dice_coefficient = np.mean(dice_coefficient_scores)
mean_pixel_accuracy = np.mean(pixel_accuracy_scores)

print("Mean IoU:", mean_iou)
print("Mean Dice Coefficient:", mean_dice_coefficient)
print("Mean Pixel Accuracy:", mean_pixel_accuracy)


1/1 [==============================] - 0s 79ms/step
Mean IoU: 1.0
Mean Dice Coefficient: 1.7129956887307596
Mean Pixel Accuracy: 0.5618018469780595


In [43]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, UpSampling2D
from tensorflow.keras.models import Model
import keras

def create_segmentation_model(input_shape, num_classes):
    # Load ResNet50 backbone
    resnet50_base = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\limuc_mob_32_model.h5")
    
    # Add segmentation-specific layers on top of ResNet50
    x = resnet50_base.layers[-4].output
    x = tf.keras.layers.UpSampling2D(size=(32, 32))(x)  # Upsample to match original image size
    x = tf.keras.layers.Conv2D(num_classes, kernel_size=(1, 1), activation='softmax')(x)

    # Create segmentation model
    model = tf.keras.models.Model(inputs=resnet50_base.input, outputs=x)
    return model

# Instantiate the model
input_shape = (32, 32, 3)  # Adjust input shape as per your dataset
num_classes = info.features['label'].num_classes
model = create_segmentation_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 16, 16, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 16, 16, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 16, 16, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 16, 16, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 16, 16, 32)       128       
 ation)                                                   

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 2, 2, 512)        2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 2, 2, 512)         0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 2, 2, 512)        4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 2, 2, 512)        2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 2, 2, 512)         0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 2, 2, 512)         262144    
                                                                 
 conv_pw_9

In [ ]:
model.fit(train_ds.batch(32), epochs=50)  # Adjust number of epochs as needed

In [ ]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(test_ds.batch(32))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
import numpy as np

# Define IoU (Intersection over Union) metric
def calculate_iou(y_true, y_pred):
    intersection = np.sum(np.logical_and(y_true, y_pred))
    union = np.sum(np.logical_or(y_true, y_pred))
    iou = intersection / union
    return iou

# Define Dice coefficient metric
def calculate_dice_coefficient(y_true, y_pred):
    intersection = 2.0 * np.sum(y_true * y_pred)
    dice_coefficient = intersection / (np.sum(y_true) + np.sum(y_pred))
    return dice_coefficient

# Define Pixel Accuracy metric
def calculate_pixel_accuracy(y_true, y_pred):
    pixel_accuracy = np.sum(y_true == y_pred) / (y_true.shape[0] * y_true.shape[1])
    return pixel_accuracy

# Evaluate the model on test data
iou_scores = []
dice_coefficient_scores = []
pixel_accuracy_scores = []

for image, label in test_ds.batch(32):
    # Predict segmentation masks
    pred_mask = model.predict(image)
    
    # Convert one-hot encoded labels back to categorical
    pred_mask_categorical = np.argmax(pred_mask, axis=-1)
    true_mask_categorical = np.argmax(label, axis=-1)
    
    # Calculate metrics for each sample
    for i in range(len(pred_mask_categorical)):
        iou_scores.append(calculate_iou(true_mask_categorical[i], pred_mask_categorical[i]))
        dice_coefficient_scores.append(calculate_dice_coefficient(true_mask_categorical[i], pred_mask_categorical[i]))
        pixel_accuracy_scores.append(calculate_pixel_accuracy(true_mask_categorical[i], pred_mask_categorical[i]))

# Calculate mean scores
mean_iou = np.mean(iou_scores)
mean_dice_coefficient = np.mean(dice_coefficient_scores)
mean_pixel_accuracy = np.mean(pixel_accuracy_scores)

print("Mean IoU:", mean_iou)
print("Mean Dice Coefficient:", mean_dice_coefficient)
print("Mean Pixel Accuracy:", mean_pixel_accuracy)
